In [1]:
import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# define stop words
stop_words = stopwords.words('english')
stop_words = sorted(stop_words)

# broardcast stop words 
broardcast_stopwords = sc.broadcast(stop_words)

def genreSentence(record):
    try:
        split_record = record.strip().split('\t')
        genre = split_record[3]
        sentence_1 = split_record[8]
        sentence_2 = split_record[9]
        return (genre,sentence_1,sentence_2)
    except:
        return()

def genreWordMatch(record):
    genre = record[0]
    sentence = record[1]+record[2]
    tokens = word_tokenize(sentence)
    words = [word for word in tokens if word.isalpha()]
    return [(genre,word.lower()) for word in words]

def removeStopWords(record):
    genre = record[0]
    word_to_check = record[1]
    found = False
    stopwords_value = broardcast_stopwords.value

    for word in stopwords_value:
        lb,ub = 0,len(stopwords_value) - 1
        
        while lb <= ub:
            mid = (lb + ub) // 2
            if stopwords_value[mid] == word_to_check:
                found = True
                break
            elif stopwords_value[mid] < word_to_check:
                lb = mid + 1
            else:
                ub = mid - 1
    if found==False:
        return [(genre,word_to_check)]
    else:
        return ()
  

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1589981080610_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Read file from storage
train = sc.textFile('s3://bgao6342/Assignment/train.tsv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Select genre, sentence1 and sentence2
train_selected = train.map(genreSentence)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Extract and remove the header
header = train_selected.first()
data_noheader = train_selected.filter(lambda x: x != header) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Map genre to each word 
genre_word_pair = data_noheader.flatMap(genreWordMatch).distinct() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Calculate the occurence of each word
word_freqency = genre_word_pair.map(lambda x: (x[1],1)).reduceByKey(lambda x,y: x+y).persist() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Count the frequency of each genre
genre_pair = word_freqency.map(lambda x: (x[1],1))
freq_count = genre_pair.countByKey().items()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Calcualte the percentage of occurrence
total_sum = 0
result = []
for x in freq_count:
    total_sum += x[1]
for x in freq_count:
    temp = x[1]
    division = str(round(x[1]/total_sum*100,4))+"%"
    result.append((x[0],division))
    
print(result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(5, '7.7255%'), (1, '69.5084%'), (4, '5.2362%'), (3, '6.4515%'), (2, '11.0784%')]

In [9]:
# Remove stop words - by using binary search 
genre_word_pair_nostop = genre_word_pair.flatMap(removeStopWords) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Calculate the occurence of each word
word_freqency_nostop = genre_word_pair_nostop.map(lambda x: (x[1],1)).reduceByKey(lambda x,y: x+y).persist() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Count the frequency of each genre
genre_pair_nostop = word_freqency_nostop.map(lambda x: (x[1],1))
freq_count_nostop = genre_pair_nostop.reduceByKey(lambda x,y:x+y)

total_sum_nostop = 0
result_nostop = []
for x in freq_count_nostop.collect():
    total_sum_nostop += x[1]
for x in freq_count_nostop.collect():
    temp = x[1]
    division = str(round(temp/total_sum_nostop*100,4))+"%"
    result_nostop.append((x[0],division))

print(result_nostop) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1, '69.6106%'), (2, '11.0901%'), (3, '6.4593%'), (4, '5.2389%'), (5, '7.601%')]